# Predicting sex from brain rhythms with deep learning

Dans un premier temps, l'objectif est de reproduire le réseau décrit dans l'article original 

https://www.nature.com/articles/s41598-018-21495-7.pdf

In [1]:
import cv2
import numpy as np
from tensorflow import keras

from imblearn.under_sampling import ClusterCentroids
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from kerastuner.tuners import RandomSearch

import sklearn
from sklearn.model_selection import train_test_split

from core.data import *

Using TensorFlow backend.


### Loading and formatting data 

In [2]:
raw_x_train, raw_y_train = load_x('data/x_train.h5'), load_y('data/y_train.csv')

Started loading file data/x_train.h5
Finished loading the file.
Started loading file data/y_train.csv
Finished loading the file.


In [3]:
# here, flatten x and convert to format (N, 7, 500, 1) and y_train in format (N, 2)

use_model_2 = True

In [4]:
h, w, c = 24, 256, 1
# if we use custom network, no need to resize data
if use_model_2:
    h, w, c = 7, 500, 1
    
def format_data(x_input, swap=False):
    """
    formatte les données pour le format de l'article original
    """
    if swap:
        x_input = swap_correlated_channels(x_input, False)
    x_flat = flatten_x(x_input)
    x_flat = reorder_nhwc(x_flat)

    # here, resize x to format 24, 256, 1

    n = len(x_flat)
    x_resized = np.zeros((n, h, w, c))

    # resize each element of x
    for i in range(n):
        x_resized[i, :, :, 0] = cv2.resize(x_flat[i, :, :, 0], dsize=(w, h))

    return x_resized

def under_sampling(X, y):
    """
    Retourne le plus grande nombre de sample tels qu'il y a autant d'hommes que de femmes
    """
    # Y doit etre categorise
    assert (len(y.shape) == 1 or (len(y.shape == 2) and y.shape[1] == 1))

    # indices where 
    men, women = np.argwhere(y == 0).squeeze(), np.argwhere(y == 1).squeeze()
    nb = min(len(men), len(women))
    men_sub_indices = np.random.choice(men, size=nb, replace=False)
    women_sub_indices = np.random.choice(women, size=nb, replace=False)
    sub_indices = np.concatenate((men_sub_indices, women_sub_indices))
    return X[sub_indices], y[sub_indices]

In [5]:
# undersample men (do not change number of women)
x, y = np.copy(raw_x_train), np.argmax(raw_y_train, axis=1)
x, y = under_sampling(x, y)
print(x.shape, y.shape)

(418, 40, 7, 500) (418,)


In [6]:
x_resized, y_resized = format_data(x), flatten_y(y, repeat=40)
x_train, x_test, y_train, y_test = train_test_split(x_resized, y_resized, test_size=0.33)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(11202, 7, 500, 1) (5518, 7, 500, 1) (11202,) (5518,)


### Definition of the model

In [7]:
# need to convert 7 x 500 into 24 x 256 ? linear interpolation + subsampling
# Hint : use openCV resize

def get_model():
    model = Sequential()
    # our model is based on convolutional NN + reLu + average pooling
    # cf cours pour la justification théorique
    model.add(layers.Conv2D(filters=100, 
                            kernel_size=(3, 3), activation='relu', data_format='channels_last', input_shape=(h, w, c)))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))

    model.add(layers.Conv2D(filters=100,
                            kernel_size=(3, 3), activation='relu', data_format='channels_last'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))

    model.add(layers.Conv2D(filters=300,
                            kernel_size=(3, 3), activation='relu', data_format='channels_last', input_shape=(h, w, c)))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))


    model.add(layers.Conv2D(filters=300, kernel_size=(1, 7), activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(1, 2)))
    model.add(layers.Dropout(rate=0.25))
    
# remove lines below to tweak architecture
    model.add(layers.Conv2D(filters=100, kernel_size=(1, 3), activation='relu'))
    model.add(layers.Conv2D(filters=100, kernel_size=(1, 3), activation='relu'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    model.compile(loss='mse', # can be binary_crossentropy or mse
              optimizer='sgd',
              metrics=['accuracy'])
    return model

In [8]:
def get_model2():
    model = Sequential()
    model.add(layers.Conv2D(filters=100, 
                            kernel_size=(h, 3), activation='relu', data_format='channels_last', input_shape=(h, w, c)))
    model.add(layers.AveragePooling2D(pool_size=(1, 2)))
    model.add(layers.Dropout(rate=0.25))
    model.add(layers.Conv2D(filters=300,
                            kernel_size=(1, 3), activation='relu', data_format='channels_last'))
    model.add(layers.AveragePooling2D(pool_size=(1, 2)))
    model.add(layers.Dropout(rate=0.25))
    model.add(layers.Conv2D(filters=300,
                            kernel_size=(1, 3), activation='relu', data_format='channels_last'))
    model.add(layers.AveragePooling2D(pool_size=(1, 2)))
    model.add(layers.Dropout(rate=0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    model.compile(loss='mse', # can be binary_crossentropy
              optimizer='sgd',
              metrics=['accuracy'])
    return model

In [9]:
class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
validation_weight = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y_test), y_test)
sample_weights = validation_weight[y_test]
print('Class weight is ', class_weight)
print('Validation weight is ', validation_weight)

Class weight is  [0.99467235 1.00538503]
Validation weight is  [1.01099304 0.98924346]


In [10]:
if use_model_2:
    print("Using model with modified convolutions")
    model = get_model2()
else:
    print('Using model w/ original convolutions')
    model = get_model()
model.fit(x_train, y_train, epochs=10, batch_size=32, class_weight=class_weight, validation_data=(x_test, y_test, sample_weights))

Using model with modified convolutions
  ...
    to  
  ['...']
Train on 11202 samples, validate on 5518 samples
Epoch 1/10


 5952/11202 [==============>...............] - ETA: 2:46 - loss: 0.3270 - accuracy: 0.62 - ETA: 1:42 - loss: 0.3510 - accuracy: 0.62 - ETA: 1:18 - loss: 0.4005 - accuracy: 0.58 - ETA: 1:06 - loss: 0.3551 - accuracy: 0.63 - ETA: 59s - loss: 0.3964 - accuracy: 0.5938 - ETA: 54s - loss: 0.4189 - accuracy: 0.572 - ETA: 51s - loss: 0.4304 - accuracy: 0.562 - ETA: 48s - loss: 0.4195 - accuracy: 0.574 - ETA: 46s - loss: 0.4284 - accuracy: 0.566 - ETA: 44s - loss: 0.4481 - accuracy: 0.546 - ETA: 43s - loss: 0.4471 - accuracy: 0.548 - ETA: 43s - loss: 0.4539 - accuracy: 0.541 - ETA: 42s - loss: 0.4502 - accuracy: 0.545 - ETA: 42s - loss: 0.4560 - accuracy: 0.540 - ETA: 41s - loss: 0.4589 - accuracy: 0.537 - ETA: 40s - loss: 0.4634 - accuracy: 0.533 - ETA: 39s - loss: 0.4691 - accuracy: 0.527 - ETA: 39s - loss: 0.4691 - accuracy: 0.527 - ETA: 38s - loss: 0.4691 - accuracy: 0.528 - ETA: 38s - loss: 0.4690 - accuracy: 0.528 - ETA: 37s - loss: 0.4720 - accuracy: 0.525 - ETA: 37s - loss: 0.4789 - ac

11202/11202 [==============================] - ETA: 15s - loss: 0.4826 - accuracy: 0.502 - ETA: 15s - loss: 0.4818 - accuracy: 0.503 - ETA: 15s - loss: 0.4813 - accuracy: 0.503 - ETA: 15s - loss: 0.4808 - accuracy: 0.503 - ETA: 15s - loss: 0.4806 - accuracy: 0.502 - ETA: 15s - loss: 0.4804 - accuracy: 0.502 - ETA: 14s - loss: 0.4802 - accuracy: 0.501 - ETA: 14s - loss: 0.4793 - accuracy: 0.502 - ETA: 14s - loss: 0.4791 - accuracy: 0.502 - ETA: 14s - loss: 0.4789 - accuracy: 0.502 - ETA: 14s - loss: 0.4782 - accuracy: 0.502 - ETA: 14s - loss: 0.4776 - accuracy: 0.502 - ETA: 14s - loss: 0.4771 - accuracy: 0.502 - ETA: 14s - loss: 0.4767 - accuracy: 0.503 - ETA: 14s - loss: 0.4775 - accuracy: 0.502 - ETA: 14s - loss: 0.4776 - accuracy: 0.501 - ETA: 14s - loss: 0.4770 - accuracy: 0.501 - ETA: 13s - loss: 0.4760 - accuracy: 0.502 - ETA: 13s - loss: 0.4761 - accuracy: 0.502 - ETA: 13s - loss: 0.4756 - accuracy: 0.503 - ETA: 13s - loss: 0.4752 - accuracy: 0.503 - ETA: 13s - loss: 0.4753 - acc

 5952/11202 [==============>...............] - ETA: 31s - loss: 0.5000 - accuracy: 0.500 - ETA: 29s - loss: 0.5312 - accuracy: 0.468 - ETA: 28s - loss: 0.5000 - accuracy: 0.500 - ETA: 28s - loss: 0.5078 - accuracy: 0.492 - ETA: 28s - loss: 0.5188 - accuracy: 0.481 - ETA: 29s - loss: 0.5260 - accuracy: 0.474 - ETA: 29s - loss: 0.5045 - accuracy: 0.495 - ETA: 28s - loss: 0.5205 - accuracy: 0.476 - ETA: 28s - loss: 0.5009 - accuracy: 0.496 - ETA: 28s - loss: 0.5195 - accuracy: 0.478 - ETA: 28s - loss: 0.5178 - accuracy: 0.480 - ETA: 28s - loss: 0.5215 - accuracy: 0.476 - ETA: 28s - loss: 0.5102 - accuracy: 0.488 - ETA: 28s - loss: 0.5117 - accuracy: 0.486 - ETA: 27s - loss: 0.5109 - accuracy: 0.487 - ETA: 27s - loss: 0.5142 - accuracy: 0.484 - ETA: 27s - loss: 0.5115 - accuracy: 0.487 - ETA: 27s - loss: 0.5143 - accuracy: 0.484 - ETA: 27s - loss: 0.5119 - accuracy: 0.486 - ETA: 27s - loss: 0.5129 - accuracy: 0.485 - ETA: 27s - loss: 0.5123 - accuracy: 0.486 - ETA: 27s - loss: 0.5089 - acc

11202/11202 [==============================] - ETA: 15s - loss: 0.5091 - accuracy: 0.490 - ETA: 15s - loss: 0.5097 - accuracy: 0.490 - ETA: 15s - loss: 0.5102 - accuracy: 0.489 - ETA: 15s - loss: 0.5098 - accuracy: 0.490 - ETA: 15s - loss: 0.5099 - accuracy: 0.489 - ETA: 15s - loss: 0.5103 - accuracy: 0.489 - ETA: 15s - loss: 0.5103 - accuracy: 0.489 - ETA: 15s - loss: 0.5107 - accuracy: 0.489 - ETA: 15s - loss: 0.5108 - accuracy: 0.488 - ETA: 15s - loss: 0.5109 - accuracy: 0.488 - ETA: 15s - loss: 0.5107 - accuracy: 0.489 - ETA: 14s - loss: 0.5102 - accuracy: 0.489 - ETA: 14s - loss: 0.5104 - accuracy: 0.489 - ETA: 14s - loss: 0.5104 - accuracy: 0.489 - ETA: 14s - loss: 0.5094 - accuracy: 0.490 - ETA: 14s - loss: 0.5092 - accuracy: 0.490 - ETA: 14s - loss: 0.5087 - accuracy: 0.491 - ETA: 14s - loss: 0.5087 - accuracy: 0.491 - ETA: 14s - loss: 0.5086 - accuracy: 0.491 - ETA: 14s - loss: 0.5089 - accuracy: 0.490 - ETA: 14s - loss: 0.5093 - accuracy: 0.490 - ETA: 14s - loss: 0.5101 - acc

...

11202/11202 [==============================] - ETA: 16s - loss: 0.5004 - accuracy: 0.499 - ETA: 16s - loss: 0.5009 - accuracy: 0.498 - ETA: 16s - loss: 0.5017 - accuracy: 0.498 - ETA: 16s - loss: 0.5022 - accuracy: 0.497 - ETA: 15s - loss: 0.5024 - accuracy: 0.497 - ETA: 15s - loss: 0.5019 - accuracy: 0.497 - ETA: 15s - loss: 0.5015 - accuracy: 0.498 - ETA: 15s - loss: 0.5019 - accuracy: 0.497 - ETA: 15s - loss: 0.5017 - accuracy: 0.498 - ETA: 15s - loss: 0.5012 - accuracy: 0.498 - ETA: 15s - loss: 0.5009 - accuracy: 0.498 - ETA: 15s - loss: 0.5006 - accuracy: 0.499 - ETA: 15s - loss: 0.5001 - accuracy: 0.499 - ETA: 15s - loss: 0.4995 - accuracy: 0.500 - ETA: 14s - loss: 0.4998 - accuracy: 0.500 - ETA: 14s - loss: 0.4995 - accuracy: 0.500 - ETA: 14s - loss: 0.4993 - accuracy: 0.500 - ETA: 14s - loss: 0.4999 - accuracy: 0.499 - ETA: 14s - loss: 0.5001 - accuracy: 0.499 - ETA: 14s - loss: 0.4996 - accuracy: 0.500 - ETA: 14s - loss: 0.4999 - accuracy: 0.499 - ETA: 14s - loss: 0.4996 - acc

 5952/11202 [==============>...............] - ETA: 34s - loss: 0.5312 - accuracy: 0.468 - ETA: 34s - loss: 0.4844 - accuracy: 0.515 - ETA: 34s - loss: 0.4792 - accuracy: 0.520 - ETA: 33s - loss: 0.4687 - accuracy: 0.531 - ETA: 33s - loss: 0.4812 - accuracy: 0.518 - ETA: 33s - loss: 0.4792 - accuracy: 0.520 - ETA: 33s - loss: 0.4598 - accuracy: 0.540 - ETA: 33s - loss: 0.4453 - accuracy: 0.554 - ETA: 33s - loss: 0.4583 - accuracy: 0.541 - ETA: 32s - loss: 0.4719 - accuracy: 0.528 - ETA: 32s - loss: 0.4687 - accuracy: 0.531 - ETA: 32s - loss: 0.4714 - accuracy: 0.528 - ETA: 32s - loss: 0.4712 - accuracy: 0.528 - ETA: 32s - loss: 0.4754 - accuracy: 0.524 - ETA: 32s - loss: 0.4729 - accuracy: 0.527 - ETA: 32s - loss: 0.4727 - accuracy: 0.527 - ETA: 32s - loss: 0.4816 - accuracy: 0.518 - ETA: 32s - loss: 0.4844 - accuracy: 0.515 - ETA: 32s - loss: 0.4934 - accuracy: 0.506 - ETA: 32s - loss: 0.4937 - accuracy: 0.506 - ETA: 32s - loss: 0.5030 - accuracy: 0.497 - ETA: 31s - loss: 0.5085 - acc

11202/11202 [==============================] - ETA: 16s - loss: 0.5019 - accuracy: 0.497 - ETA: 16s - loss: 0.5017 - accuracy: 0.498 - ETA: 16s - loss: 0.5022 - accuracy: 0.497 - ETA: 16s - loss: 0.5019 - accuracy: 0.497 - ETA: 16s - loss: 0.5024 - accuracy: 0.497 - ETA: 15s - loss: 0.5030 - accuracy: 0.496 - ETA: 15s - loss: 0.5030 - accuracy: 0.496 - ETA: 15s - loss: 0.5027 - accuracy: 0.497 - ETA: 15s - loss: 0.5022 - accuracy: 0.497 - ETA: 15s - loss: 0.5028 - accuracy: 0.497 - ETA: 15s - loss: 0.5018 - accuracy: 0.497 - ETA: 15s - loss: 0.5021 - accuracy: 0.497 - ETA: 15s - loss: 0.5026 - accuracy: 0.497 - ETA: 15s - loss: 0.5026 - accuracy: 0.497 - ETA: 15s - loss: 0.5018 - accuracy: 0.498 - ETA: 14s - loss: 0.5018 - accuracy: 0.498 - ETA: 14s - loss: 0.5018 - accuracy: 0.498 - ETA: 14s - loss: 0.5019 - accuracy: 0.497 - ETA: 14s - loss: 0.5021 - accuracy: 0.497 - ETA: 14s - loss: 0.5022 - accuracy: 0.497 - ETA: 14s - loss: 0.5022 - accuracy: 0.497 - ETA: 14s - loss: 0.5017 - acc

 5952/11202 [==============>...............] - ETA: 31s - loss: 0.4688 - accuracy: 0.531 - ETA: 32s - loss: 0.5312 - accuracy: 0.468 - ETA: 32s - loss: 0.5625 - accuracy: 0.437 - ETA: 32s - loss: 0.5312 - accuracy: 0.468 - ETA: 32s - loss: 0.5625 - accuracy: 0.437 - ETA: 32s - loss: 0.5677 - accuracy: 0.432 - ETA: 32s - loss: 0.5759 - accuracy: 0.424 - ETA: 32s - loss: 0.5508 - accuracy: 0.449 - ETA: 32s - loss: 0.5521 - accuracy: 0.447 - ETA: 32s - loss: 0.5531 - accuracy: 0.446 - ETA: 32s - loss: 0.5455 - accuracy: 0.454 - ETA: 32s - loss: 0.5443 - accuracy: 0.455 - ETA: 32s - loss: 0.5312 - accuracy: 0.468 - ETA: 31s - loss: 0.5268 - accuracy: 0.473 - ETA: 31s - loss: 0.5229 - accuracy: 0.477 - ETA: 32s - loss: 0.5293 - accuracy: 0.470 - ETA: 32s - loss: 0.5423 - accuracy: 0.457 - ETA: 32s - loss: 0.5469 - accuracy: 0.453 - ETA: 31s - loss: 0.5444 - accuracy: 0.455 - ETA: 31s - loss: 0.5453 - accuracy: 0.454 - ETA: 31s - loss: 0.5417 - accuracy: 0.458 - ETA: 31s - loss: 0.5440 - acc

11202/11202 [==============================] - ETA: 16s - loss: 0.5054 - accuracy: 0.494 - ETA: 16s - loss: 0.5061 - accuracy: 0.493 - ETA: 16s - loss: 0.5065 - accuracy: 0.493 - ETA: 16s - loss: 0.5067 - accuracy: 0.493 - ETA: 16s - loss: 0.5073 - accuracy: 0.492 - ETA: 16s - loss: 0.5061 - accuracy: 0.493 - ETA: 16s - loss: 0.5056 - accuracy: 0.494 - ETA: 16s - loss: 0.5048 - accuracy: 0.495 - ETA: 15s - loss: 0.5054 - accuracy: 0.494 - ETA: 15s - loss: 0.5050 - accuracy: 0.494 - ETA: 15s - loss: 0.5055 - accuracy: 0.494 - ETA: 15s - loss: 0.5047 - accuracy: 0.495 - ETA: 15s - loss: 0.5054 - accuracy: 0.494 - ETA: 15s - loss: 0.5045 - accuracy: 0.495 - ETA: 15s - loss: 0.5044 - accuracy: 0.495 - ETA: 15s - loss: 0.5038 - accuracy: 0.496 - ETA: 15s - loss: 0.5039 - accuracy: 0.495 - ETA: 15s - loss: 0.5041 - accuracy: 0.495 - ETA: 14s - loss: 0.5042 - accuracy: 0.495 - ETA: 14s - loss: 0.5045 - accuracy: 0.495 - ETA: 14s - loss: 0.5043 - accuracy: 0.495 - ETA: 14s - loss: 0.5043 - acc

In [11]:
raw_x_test = load_x('data/x_test.h5')

Started loading file data/x_test.h5
Finished loading the file.


In [12]:
x_test0 = format_data(raw_x_test)

raw_predictions = model.predict(x_test0)
y_test0 = average_predictions(raw_predictions, nb_trials = 40)
print(y_test0)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [13]:
save_csv(y_test0, 'data/result_undersampling_natureNN.csv')

In [14]:
print(raw_x_test.shape)
print(x_test0.shape)

(946, 40, 7, 500)
(37840, 7, 500, 1)
